In [ ]:
# avg.sim (Figure 1)

import json
import os
from utility.load_data import Data

data_generator = Data(path='data/WomenClothing', batch_size=1024)

from copy import deepcopy
I_items = deepcopy(data_generator.train_items)

for k in I_items.keys():
    I_items[k] = I_items[k] + data_generator.val_set[k] + data_generator.test_set[k]

import numpy as np
image_feats = np.load('data/WomenClothing/image_feat.npy')
text_feats = np.load('data/WomenClothing/text_feat.npy')

from collections import defaultdict
from tqdm import tqdm

img_cos = np.dot(image_feats, image_feats.T) / (np.linalg.norm(image_feats, axis=1)[:, np.newaxis] * np.linalg.norm(image_feats, axis=1)[:, np.newaxis].T)
text_cos = np.dot(text_feats, text_feats.T) / (np.linalg.norm(text_feats, axis=1)[:, np.newaxis] * np.linalg.norm(text_feats, axis=1)[:, np.newaxis].T)

seen_img = []
seen_text = []
unseen_img = []
unseen_text = []
for user, items in tqdm(I_items.items()):
    img = img_cos[items][:, items]
    text = text_cos[items][:, items]

    seen_img_result = []
    seen_text_result = []
    for i in range(len(items)):
        seen_img_result.append(np.concatenate([img[i, :i], img[i, i+1:]]))
        seen_text_result.append(np.concatenate([text[i, :i], text[i, i+1:]]))
    seen_img_result = np.array(seen_img_result) # .flatten()
    seen_text_result = np.array(seen_text_result) # .flatten()

    unseen_items = set(range(data_generator.n_items)) - set(items)
    unseen_items = list(unseen_items)

    unseen_img_result = img_cos[items][:, unseen_items].flatten()
    unseen_text_result = text_cos[items][:, unseen_items].flatten()

    seen_img.append(seen_img_result.mean())
    seen_text.append(seen_text_result.mean())
    unseen_img.append(unseen_img_result.mean())
    unseen_text.append(unseen_text_result.mean())

print('text', np.mean(seen_text), np.mean(unseen_text))
print('img', np.mean(seen_img), np.mean(unseen_img))

n_users=19244, n_items=14596
n_interactions=135326
n_train=95629, n_val=20127, n_test=19570, sparsity=0.00048


100%|██████████| 19244/19244 [01:10<00:00, 273.56it/s]

text 0.10991018 0.086630285
img 0.3009104 0.23256181


In [1]:
# avg.diff
from Models import *

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import torch
import numpy as np
from utility.load_data import Data
data_generator = Data(path='data/WomenClothing', batch_size=1024)

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

model_name = 'MONET_concat_20_03'
seed_list = ['123', '0', '42', '1024', '2048']
nonzero_idx = data_generator.nonzero_idx()

import numpy as np
image_feats = np.load('data/WomenClothing/image_feat.npy')
text_feats = np.load('data/WomenClothing/text_feat.npy')
    
for seed in seed_list:    
    model = MONET(data_generator.n_users, data_generator.n_items, 64, nonzero_idx, True, image_feats, text_feats, 2, 1.0, 0.3, 'concat', 's', False)    
    model.load_state_dict(torch.load('./models/' + 'WomenClothing' + '_' + model_name + '_' + seed, map_location=torch.device('cpu'))[model_name + '_' + seed])
    model.cuda()
    image_emb, text_emb = model(eval=True)
    print('Loads image_emb: {} and text_emb: {}'.format(image_emb.shape, text_emb.shape))

    # user_emb = torch.load('data/{}/{}_user_emb.pt'.format('clothing', 'lightgcn_layer3_original')).cuda()
    # item_emb = torch.load('data/{}/{}_item_emb.pt'.format('clothing', 'lightgcn_layer3_original')).cuda()
    # print('Loads user_emb: {} and item_emb: {}'.format(user_emb.weight.shape, item_emb.weight.shape))

    # image_emb = image_emb.mean(dim=1, keepdim=False)
    # text_emb = text_emb.mean(dim=1, keepdim=False)

    # image_emb = image_emb[:, -1, :]
    # text_emb = text_emb[:, -1, :]


    final_image_preference, final_image_emb = torch.split(image_emb, [data_generator.n_users, data_generator.n_items], dim=0)
    final_text_preference, final_text_emb = torch.split(text_emb, [data_generator.n_users, data_generator.n_items], dim=0)

    final_text_emb, final_image_emb = final_text_emb.cpu().detach().numpy(), final_image_emb.cpu().detach().numpy()

    final_image_cos = np.dot(final_image_emb, final_image_emb.T) / (np.linalg.norm(final_image_emb, axis=1)[:, np.newaxis] * np.linalg.norm(final_image_emb, axis=1)[:, np.newaxis].T)
    final_text_cos = np.dot(final_text_emb, final_text_emb.T) / (np.linalg.norm(final_text_emb, axis=1)[:, np.newaxis] * np.linalg.norm(final_text_emb, axis=1)[:, np.newaxis].T)

    img_cos = np.dot(image_feats, image_feats.T) / (np.linalg.norm(image_feats, axis=1)[:, np.newaxis] * np.linalg.norm(image_feats, axis=1)[:, np.newaxis].T)
    text_cos = np.dot(text_feats, text_feats.T) / (np.linalg.norm(text_feats, axis=1)[:, np.newaxis] * np.linalg.norm(text_feats, axis=1)[:, np.newaxis].T)

    img_diff = np.abs(img_cos - final_image_cos)
    text_diff = np.abs(text_cos - final_text_cos)

    img = []
    for i in range(data_generator.n_items):
        img.append(np.concatenate([img_diff[i, :i], img_diff[i, i+1:]]))
    img = np.array(img) # .flatten()

    txt = []
    for i in range(data_generator.n_items):
        txt.append(np.concatenate([text_diff[i, :i], text_diff[i, i+1:]]))
    txt = np.array(txt) # .flatten()

    print(img[~np.isnan(img)].mean(), txt[~np.isnan(txt)].mean())

/home/ubuntu/anaconda3/envs/yg/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


n_users=19244, n_items=14596
n_interactions=135326
n_train=95629, n_val=20127, n_test=19570, sparsity=0.00048
Loads image_emb: torch.Size([33840, 64]) and text_emb: torch.Size([33840, 64])
0.14228745 0.11385205
Loads image_emb: torch.Size([33840, 64]) and text_emb: torch.Size([33840, 64])
0.3034921 0.10677319
Loads image_emb: torch.Size([33840, 64]) and text_emb: torch.Size([33840, 64])
0.3312145 0.1141393
Loads image_emb: torch.Size([33840, 64]) and text_emb: torch.Size([33840, 64])
0.17027126 0.110791825
Loads image_emb: torch.Size([33840, 64]) and text_emb: torch.Size([33840, 64])
0.2663411 0.11235878
